# Aaron's ISR challenge - are amp boundaries visible after ISR?
Craig Lage 09-Jan-23

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm

from lsst.summit.utils import quickSmooth
%matplotlib inline

from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask

In [ ]:
def plotExp(exp, expId):
    # Borrowed from summit utils
    data = quickSmooth(exp.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    ax1.set_title(f"{expId}")
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure, vmin, vmax

def plotArr(arr):
    # Borrowed from summit utils
    data = quickSmooth(arr, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    #ax1.set_title(f"{expId}")
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    #plt.show()
    return figure, vmin, vmax

def repairCosmics(postIsr):
    # Borrowed from summit utils
    if postIsr.getPsf() is None:
        installPsfTask = InstallGaussianPsfTask()
        installPsfTask.run(postIsr)

    # TODO: try adding a reasonably wide Gaussian as a temp PSF
    # and then just running repairTask on its own without any
    # imChar. It should work, and be faster.
    repairConfig = CharacterizeImageTask.ConfigClass()
    repairConfig.doMeasurePsf = False
    repairConfig.doApCorr = False
    repairConfig.doDeblend = False
    repairConfig.doWrite = False
    repairConfig.repair.cosmicray.nCrPixelMax = 200000
    repairTask = CharacterizeImageTask(config=repairConfig)
    repairTask.repair.run(postIsr)
    return

def fitPlane(exp):
    x = np.arange(0,exp.image.array.shape[0])
    y = np.arange(0,exp.image.array.shape[1])
    X, Y = np.meshgrid(y, x)
    xRavel, yRavel = X.ravel(), Y.ravel()
    basis = []
    order = 1
    for i in range(order+1):
        for j in range(order - i +1):
            basis.append(xRavel**j * yRavel**i)
    A = np.vstack(basis).T
    b = exp.image.array.ravel()
    c, r, rank, s = np.linalg.lstsq(A, b, rcond=None)
    plane = c[0] + c[1] * X + c[2] * Y
    return plane
 
def correctGains(exp):
    plane = fitPlane(exp)
    xCut = 50
    yCut = 500
    corrs = {}
    ratioSum = 0
    for amp in exp.getDetector().getAmplifiers():
        ampName = amp.getName()
        bbox = amp.getBBox()
        xmin = bbox.beginX + xCut
        xmax = bbox.endX - xCut
        ymin = bbox.beginY + yCut
        ymax = bbox.endY - yCut
        imMedian = np.median(exp.image.array[ymin:ymax,xmin:xmax])
        planeMedian = np.median(plane[ymin:ymax,xmin:xmax])
        ratio = planeMedian / imMedian
        ratioSum += ratio
        corrs[ampName] = ratio
    factor = ratioSum / 16.0
    for amp in exp.getDetector().getAmplifiers():
        ampName = amp.getName()
        corrs[ampName] /= factor
        print(ampName, corrs[ampName])
    return corrs
    
    
    
def correctGainsOld(exp):
    xskip = 20
    yskip = 100
    xfitVals = (1000, 3000)
    corrs = {}
    xfit = np.arange(xfitVals[0], xfitVals[1]) # Only fit the middle amps
    for amp in exp.getDetector().getAmplifiers():
        xmin = amp.getBBox().beginX + xskip
        xmax = amp.getBBox().endX - xskip
        ymin = amp.getBBox().beginY + yskip
        ymax = amp.getBBox().endY - yskip
        xvals = np.median(exp.image.array[ymin:ymax, 0:4000], axis=0)
        fit = np.polyfit(xfit, xvals[xfitVals[0]:xfitVals[1]], 1)
        xplot = np.arange(0, 4000)
        yplot = xplot * fit[0] + fit[1]
        corr = np.median((xvals/yplot)[xmin:xmax])
        print(amp.getName(), corr)
        #plt.plot(xvals, label="PostISR")
        #plt.plot(xplot, yplot, color='red', ls='--')
        #plt.ylim(1400, 1520)
        #break
        corrs[amp.getName()] = corr
    # Vertical correction
    # This needs more work
    """
    yvals = np.median(isrResult.exposure.image.array[0:4000, 100:3900], axis=1)
    yfitVals = (500, 1900)
    ycalcVals = (2100, 3000)
    yfit = np.arange(yfitVals[0], yfitVals[1])
    fit = np.polyfit(yfit, yvals[yfitVals[0]:yfitVals[1]], 1)
    xplot = np.arange(0, 4000)
    yplot = xplot * fit[0] + fit[1]
    verticalCorr = np.median((yvals/yplot)[ycalcVals[0]:ycalcVals[1]])
    for amp in exp.getDetector().getAmplifiers():
        ymin = amp.getBBox().beginY + yskip
        if ymin > 1000.0:
            corrs[amp.getName()] *= verticalCorr
    """        
    return corrs


In [ ]:
butler = Butler("/repo/main", collections=["LSSTCam/raw/all","u/cslage/calib/13144/calib.20220107"])
biasButler = Butler("/repo/main", collections=["u/cslage/bias_parallel_13144_B_17jan23"])

In [ ]:
expId = 3021120700773
detector = 55

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doLinearize=True
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doBrighterFatter=False
isrConfig.doDark=True
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=True
isrConfig.doDefect=True
isrConfig.doDeferredCharge=True
isrConfig.doNanMasking=True
isrConfig.doInterpolate=True
isrConfig.doSaturation=True
isrConfig.doSaturationInterpolation=True
isrConfig.doWrite=False
isrTask = IsrTask(config=isrConfig)

## Run first to get the gain corrections

In [ ]:
# Zero the gain correction
exp = butler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")
corrs = {}
for amp in exp.getDetector().getAmplifiers():
    corrs[amp.getName()] = 1.0

In [ ]:
exp = butler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")
bias = biasButler.get('bias', detector=detector, exposure=expId, instrument="LSSTCam")
flat = butler.get('flat', detector=detector, exposure=expId, instrument="LSSTCam")
dark = butler.get('dark', detector=detector, exposure=expId, instrument="LSSTCam")
defects = butler.get('defects', detector=detector, exposure=expId, instrument="LSSTCam")
ptc = butler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
linearizer = butler.get('linearizer', detector=detector, exposure=expId, instrument="LSSTCam")
deferredChargeCalib = butler.get('cpCtiCalib', detector=detector, exposure=expId, instrument="LSSTCam")


isrResult = isrTask.run(exp, bias=bias, flat=flat, dark=dark, defects=defects, \
                        ptc=ptc, linearizer=linearizer, deferredChargeCalib=deferredChargeCalib)

postIsrExp = isrResult.exposure
#repairCosmics(postIsrExp)
fig, vmin, vmax = plotExp(postIsrExp, expId)
#plt.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_{detector}_{expId}_17Jan23.png")

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)
rawPlot = np.median(isrResult.exposure.image.array[2100:3900, 0:4000], axis=0)
xplot1 = np.arange(520, 3500)
vals = np.polyfit(xplot1, rawPlot[520:3500], 1)
xplot2 = np.arange(0, 4000)
yplot = xplot2 * vals[0] + vals[1]
plt.subplot(2,1,1)
plt.title(f"Median Flat Level - Top - {expId}")
plt.plot(rawPlot, label="PostISR")
plt.plot(xplot2, yplot, color='red', ls='--', lw=2, label="Fit")
for i in range(8):
    x = 509 * i
    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
xtext = 4200
ytext = vmax
plt.text(xtext, ytext, "Gain adjustments:")
for amp in exp.getDetector().getAmplifiers():
    ytext -= vmax/100
    plt.text(xtext, ytext, f"{amp.getName()}: {(corrs[amp.getName()] - 1.0) * 100.0:.2f}%")
rawPlot = np.median(isrResult.exposure.image.array[100:1900, 0:4000], axis=0)
xplot1 = np.arange(520, 3500)
vals = np.polyfit(xplot1, rawPlot[520:3500], 1)
xplot2 = np.arange(0, 4000)
yplot = xplot2 * vals[0] + vals[1]
plt.subplot(2,1,2)
plt.title(f"Median Flat Level - Bottom - {expId}")
plt.plot(rawPlot, label="PostISR")
plt.plot(xplot2, yplot, color='red', ls='--', lw=2, label="Fit")
for i in range(8):
    x = 509 * i
    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

#fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_{detector}_{expId}_Scan_17Jan23.png")

In [ ]:
plane = fitPlane(postIsrExp)

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)
rawPlot = np.median(isrResult.exposure.image.array[2100:3900, 0:4000], axis=0)
planePlot = np.median(plane[2100:3900, 0:4000], axis=0)
plt.subplot(2,1,1)
plt.title(f"Median Flat Level - Top - {expId}")
plt.plot(rawPlot, label="PostISR")
plt.plot(planePl, color='red', ls='--', lw=2, label="Fit")
for i in range(8):
    x = 509 * i
    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
xtext = 4200
ytext = vmax
plt.text(xtext, ytext, "Gain adjustments:")
for amp in exp.getDetector().getAmplifiers():
    ytext -= vmax/100
    plt.text(xtext, ytext, f"{amp.getName()}: {(corrs[amp.getName()] - 1.0) * 100.0:.2f}%")
rawPlot = np.median(isrResult.exposure.image.array[100:1900, 0:4000], axis=0)
xplot1 = np.arange(520, 3500)
vals = np.polyfit(xplot1, rawPlot[520:3500], 1)
xplot2 = np.arange(0, 4000)
yplot = xplot2 * vals[0] + vals[1]
plt.subplot(2,1,2)
plt.title(f"Median Flat Level - Bottom - {expId}")
plt.plot(rawPlot, label="PostISR")
plt.plot(xplot2, yplot, color='red', ls='--', lw=2, label="Fit")
for i in range(8):
    x = 509 * i
    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

#fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_{detector}_{expId}_Scan_17Jan23.png")

In [ ]:
# Try with the plane code
x = np.arange(0,postIsrExp.image.array.shape[0])
y = np.arange(0,postIsrExp.image.array.shape[1])
X, Y = np.meshgrid(y, x)
xRavel, yRavel = X.ravel(), Y.ravel()
basis = []
order = 1
for i in range(order+1):
    for j in range(order - i +1):
        basis.append(xRavel**j * yRavel**i)


In [ ]:
def fitPlane(arr, basis):


In [ ]:
def fitPlane(exp):
    x = np.arange(0,exp.image.array.shape[0])
    y = np.arange(0,exp.image.array.shape[1])
    X, Y = np.meshgrid(y, x)
    xRavel, yRavel = X.ravel(), Y.ravel()
    basis = []
    order = 1
    for i in range(order+1):
        for j in range(order - i +1):
            basis.append(xRavel**j * yRavel**i)

    A = np.vstack(basis).T
    b = arr.ravel()
    c, r, rank, s = np.linalg.lstsq(A, b, rcond=None)
    plane = c[0] + c[1] * X + c[2] * Y
    return plane
    

In [ ]:
c = fitPlane(postIsrExp.image.array, basis)

In [ ]:

plotArr(plane)

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)
rawPlot = np.median(postIsrExp.image.array[3000:3200, 0:4000], axis=0)
xplot1 = np.arange(520, 3500)
vals = np.polyfit(xplot1, rawPlot[520:3500], 1)
xplot2 = np.arange(0, 4000)
#yplot = xplot2 * vals[0] + vals[1]
yplot = c[0] + c[1] * xplot2 + c[2] * 3100.0
plt.subplot(2,1,1)
plt.title(f"Median Flat Level - Top - {expId}")
plt.plot(rawPlot, label="PostISR")
plt.plot(xplot2, yplot, color='red', ls='--', lw=2, label="Fit")
for i in range(8):
    x = 509 * i
    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
xtext = 4200
ytext = vmax
plt.text(xtext, ytext, "Gain adjustments:")
for amp in exp.getDetector().getAmplifiers():
    ytext -= vmax/100
    plt.text(xtext, ytext, f"{amp.getName()}: {(corrs[amp.getName()] - 1.0) * 100.0:.2f}%")
rawPlot = np.median(postIsrExp.image.array[1000:1200, 0:4000], axis=0)
xplot1 = np.arange(520, 3500)
vals = np.polyfit(xplot1, rawPlot[520:3500], 1)
xplot2 = np.arange(0, 4000)
#yplot = xplot2 * vals[0] + vals[1]
yplot = c[0] + c[1] * xplot2 + c[2] * 1100.0
plt.subplot(2,1,2)
plt.title(f"Median Flat Level - Bottom - {expId}")
plt.plot(rawPlot, label="PostISR")
plt.plot(xplot2, yplot, color='red', ls='--', lw=2, label="Fit")
for i in range(8):
    x = 509 * i
    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

#fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_{detector}_{expId}_Scan_17Jan23.png")

In [ ]:
plane = fitPlane(postIsrExp)

In [ ]:
xCut = 50
yCut = 500
corrs_new = {}
for amp in postIsrExp.getDetector().getAmplifiers():
    ampName = amp.getName()
    bbox = amp.getBBox()
    xmin = bbox.beginX + xCut
    xmax = bbox.endX - xCut
    ymin = bbox.beginY + yCut
    ymax = bbox.endY - yCut
    imMedian = np.median(postIsrExp.image.array[ymin:ymax,xmin:xmax])
    planeMedian = np.median(plane[ymin:ymax,xmin:xmax])
    ratio = imMedian / planeMedian
    print(ampName, ratio)
    corrs_new[ampName] = ratio

In [ ]:
bbox.

In [ ]:
np.sum((postIsrExp.image.array - plane) * (postIsrExp.image.array - plane))

In [ ]:
clone = postIsrExp.clone()
for amp in clone.getDetector().getAmplifiers(): 
    clone[amp.getBBox()].image.array /= corrs_old[amp.getName()]
c = fitPlane(clone.image.array, basis)
plane = c[0] + c[1] * X + c[2] * Y # Inverted because it always is.

In [ ]:
np.sum((clone.image.array - plane) * (clone.image.array - plane))

## Now correct the gains and run again.

In [ ]:
corrs = correctGains(postIsrExp)

In [ ]:
sum = 0.0
for amp in postIsrExp.getDetector().getAmplifiers():
    ampName = amp.getName()
    sum += corrs[ampName]
print(sum / 16.0)


In [ ]:
exp = butler.get('raw', detector=detector, exposure=expId, instrument="LSSTCam")
bias = biasButler.get('bias', detector=detector, exposure=expId, instrument="LSSTCam")
flat = butler.get('flat', detector=detector, exposure=expId, instrument="LSSTCam")
dark = butler.get('dark', detector=detector, exposure=expId, instrument="LSSTCam")
defects = butler.get('defects', detector=detector, exposure=expId, instrument="LSSTCam")
ptc = butler.get('ptc', detector=detector, exposure=expId, instrument="LSSTCam")
for amp in exp.getDetector().getAmplifiers():
    #ptc.gain[amp.getName()] /= corrs_new[amp.getName()]
    ptc.gain[amp.getName()] *= corrs[amp.getName()]
linearizer = butler.get('linearizer', detector=detector, exposure=expId, instrument="LSSTCam")
deferredChargeCalib = butler.get('cpCtiCalib', detector=detector, exposure=expId, instrument="LSSTCam")


isrResult_new = isrTask.run(exp, bias=bias, flat=flat, dark=dark, defects=defects, \
                        ptc=ptc, linearizer=linearizer, deferredChargeCalib=deferredChargeCalib)

postIsrExp_new = isrResult_new.exposure
#repairCosmics(postIsrExp)
fig, vmin, vmax = plotExp(postIsrExp_new, expId)
#plt.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_Gain_Corrected_{detector}_{expId}_17Jan23.png")#

In [ ]:
fig = plt.figure()
plt.subplots_adjust(hspace=0.5, wspace=0.5)
rawPlot = np.median(isrResult.exposure.image.array[2100:3900, 0:4000], axis=0)
xplot1 = np.arange(520, 3500)
vals = np.polyfit(xplot1, rawPlot[520:3500], 1)
xplot2 = np.arange(0, 4000)
yplot = xplot2 * vals[0] + vals[1]
plt.subplot(2,1,1)
plt.title(f"Median Flat Level - Top - {expId}")
plt.plot(rawPlot, label="PostISR")
plt.plot(xplot2, yplot, color='red', ls='--', lw=2, label="Fit")
for i in range(8):
    x = 509 * i
    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
xtext = 4200
ytext = vmax
plt.text(xtext, ytext, "Gain adjustments:")
for amp in exp.getDetector().getAmplifiers():
    ytext -= vmax/100
    plt.text(xtext, ytext, f"{amp.getName()}: {(corrs[amp.getName()] - 1.0) * 100.0:.2f}%")
rawPlot = np.median(isrResult.exposure.image.array[100:1900, 0:4000], axis=0)
xplot1 = np.arange(520, 3500)
vals = np.polyfit(xplot1, rawPlot[520:3500], 1)
xplot2 = np.arange(0, 4000)
yplot = xplot2 * vals[0] + vals[1]
plt.subplot(2,1,2)
plt.title(f"Median Flat Level - Bottom - {expId}")
plt.plot(rawPlot, label="PostISR")
plt.plot(xplot2, yplot, color='red', ls='--', lw=2, label="Fit")
for i in range(8):
    x = 509 * i
    plt.plot([x, x], [vmin, vmax], ls = '--', lw=1, color = 'black')
plt.xlim(0,4000)
plt.ylim(vmin, vmax)
plt.legend()
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")

#fig.savefig(f"/home/c/cslage/u/BOT_LSSTCam/isr_challenge/Challenge_Gain_Corrected_{detector}_{expId}_Scan_17Jan23.png")

In [ ]:
# We're also going to need a top/bottom correction
yvals = np.median(isrResult.exposure.image.array[0:4000, 100:3900], axis=1)
plt.plot(yvals)
yfitVals = (500, 1900)
ycalcVals = (2100, 3000)
yfit = np.arange(yfitVals[0], yfitVals[1])
fit = np.polyfit(yfit, yvals[yfitVals[0]:yfitVals[1]], 1)
xplot = np.arange(0, 4000)
yplot = xplot * fit[0] + fit[1]
plt.plot(xplot, yplot)
plt.ylim(74000, 76000)
verticalCorr = np.median((yvals/yplot)[ycalcVals[0]:ycalcVals[1]])
print(verticalCorr)

In [ ]:
from scipy.optimize import minimize

def fitPlane(arr, basis):
    A = np.vstack(basis).T
    b = arr.ravel()
    c, r, rank, s = np.linalg.lstsq(A, b, rcond=None)
    return c

def calcError(corrs, args):
    [exp, basis, X, Y] = args
    # Multiply each amp by the gain correction factor
    clone = exp.clone()
    ampCounter = 0
    for amp in clone.getDetector().getAmplifiers(): 
        if amp.getName() == 'C04':
            continue
        clone[amp.getBBox()].image.array *= corrs[ampCounter]
        ampCounter += 1
    # Now fit the flux plane
    arr = clone.image.array
    c = fitPlane(arr, basis)
    # Now calculate the least square error between the plane and the array
    plane = c[0] + c[1] * X + c[2] * Y 
    error = np.sum((arr - plane) * (arr - plane))
    print(f"Error = {error:.5g}")
    return error


def correctGains(exp):
    minCorr = 0.99
    maxCorr = 1.01
    param0=[]
    bounds=[]
    for i in range(15):
        param0.append(1.0)
        bounds.append((minCorr, maxCorr))
    x = np.arange(0,exp.image.array.shape[0])
    y = np.arange(0,exp.image.array.shape[1])
    X, Y = np.meshgrid(y, x)
    xRavel, yRavel = X.ravel(), Y.ravel()
    basis = []
    order = 1
    for i in range(order+1):
        for j in range(order - i +1):
            basis.append(xRavel**j * yRavel**i)
    args = [exp, basis, X, Y]
    result = minimize(calcError, param0, args=args, method='Powell', bounds=bounds)
    print(result)
    corrs = {}
    ampCounter = 0
    for amp in exp.getDetector().getAmplifiers(): 
        ampName = amp.getName()
        if ampName == 'C04':
            corrs[ampName] = 1.0
        else:
            corrs[ampName] = result.x[ampCounter]
            ampCounter += 1
    return [corrs, result]


In [ ]:
[corrs, result] = correctGains(postIsrExp)

In [ ]:
corrs

In [ ]:
corrs_old = correctGainsOld(postIsrExp)

In [ ]:
for amp in exp.getDetector().getAmplifiers(): 
    ampName = amp.getName()
    print(ampName, corrs[ampName], 1.0/corrs_old[ampName]*corrs_old['C04'])